In [89]:
words = ["abogado","aceite","áfrica","agente","agua","águila","aguja","agujero","aire","alemania","algodón","alianza","alpes","ambulancia","américa","ángel","anillo","antártida","antorcha","araña","archivo","arco","argentina","artículo","as","atlántida","azteca","baile","bala","ballena","banco","banda","baño","barco","barra","batería","berlín","bermudas","bicho","blanco","bloque","boca","bola","bolsa","bomba","bosque","bota","botella","botón","brazo","bruja","caballero","caballo","cabeza","cabina","cabo","cactus","cadena","caja","cama","cámara","cambio","campana","campo","canal","canguro","canto","caña","capa","capital","caqui","cara","caravana","carga","carrera","carro","carta","casco","casino","caza","cementerio","centauro","centro","cervantes","checo","chocolate","choque","chuleta","científico","cinta","cinturón","círculo","clase","coche","cocinero","coco","código","cola","cólera","columna","cometa","compás","concierto","conejo","contrabandista","copa","corazón","corneta","corona","corredor","corriente","corte","cresta","cromo","cruz","cuadro","cuarto","cubierta","cubo","cuchillo","cuello","cuerda","cuerno","cura","dama","delta","destino","día","diamante","diana","diario","diente","dinosaurio","disco","don","dragón","duende","egipto","embajada","emperador","enano","enfermedad","enfermera","enlace","escorpión","espacio","espía","estación","estadio","estado","estrella","estudio","etiqueta","europa","extraterrestre","falda","fantasma","faro","ficha","fiesta","figura","flauta","flecha","foso","francia","frente","fuego","fuente","fuerza","furgoneta","gancho","gato","genio","gigante","golfo","golondrina","golpe","goma","góndola","gota","grado","granada","grano","grecia","grifo","guante","guardia","guerra","gusano","helado","helicóptero","hielo","hierba","hoja","hollywood","horca","hospital","hotel","iglesia","imán","india","índice","inglaterra","italia","jarra","judía","juicio","kiwi","ladrón","lago ness","láser","látigo","lengua","león","libra","lima","limusina","línea","lista","llama","llave","lomo","londres","luna","luz","maestro","magia","malta","mancha","mando","manga","mango","mano","manzana","mañana","marca","marcha","marfil","masa","máscara","mazo","médico","mercurio","mesa","metro","méxico","micro","microscopi","mielo","millonario","mina","misil","modelo","módulo","monitor","mono","mortero","moscú","motor","muelle","muerte","muñeca","muro","naranja","nave","nieve","nilo","ninja","noche","nota","nudo","nueva york","obra","ojo","ola","olimpo","ópera","orden","órgano","ornitorrinco","oro","oso","pala","palma","pantalla","papel","paracaídas","pase","paso","pasta","pastel","pavo","pekín","película","pelotón","pendiente","perro","pez","pico","pie","pieza","pila","piloto","pincho","pingüino","pinta","piña","pirámide","pirata","pista","pistola","placa","plano","planta","plátano","playa","plomo","pluma","policía","polo","portada","portero","potro","prensa","prima","princesa","puente","puerto","pulpo","pulso","punta","punto","radio","rascacielos","ratón","rayo","red","regla","reina","reserva","revolución","rey","robot","rojo","roma","ronda","rosa","ruleta","sable","sáhara","salsa","satélite","saturno","señal","serie","serpiente","sierra","silla","sirena","sobre","soldado","submarinista","suerte","superhéroe","tabla","tableta","taco","tacto","talón","tanque","tapa","tarde","teatro","teclado","telescopio","testigo","tiempo","tienda","tierra","tokio","topo","torre","trama","tronco","tubería","tubo","unicornio","vacío","vado","vampiro","vela","veneno","venus","vestido","vida","vidrio","viento","yema","zanahoria","zapato"]

In [91]:
import fasttext
import fasttext.util
import gensim
from gensim.models import FastText, Word2Vec
from gensim.models.fasttext import load_facebook_model
from nltk.corpus import wordnet as wn
import nltk
import itertools
import pandas as pd

In [2]:
nltk.download("omw-1.4")
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\josem\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\josem\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [127]:
def get_cosine_similarity(model, word1, word2):
    return model.wv.similarity(word1, word2)

def get_synonyms(word1, lang = 'spa'):
    all_synonyms = list(map(lambda x: x.lemma_names(lang), wn.synsets(word1, lang=lang)))
    return list(set(itertools.chain(*all_synonyms)))

def get_antonyms(word1, lang = 'spa'):
    all_lemas = list(map(lambda x: x.lemmas(), wn.synsets(word1, lang=lang)))
    all_antonyms_lemas = list(map(lambda x: x.antonyms(), list(itertools.chain(*all_lemas))))
    all_antonyms = map(lambda x: x.synset().lemma_names(lang=lang), list(itertools.chain(*all_antonyms_lemas)))
    return list(set(itertools.chain(*all_antonyms)))

def get_synonyms_with_similarity(word1, model, lang='spa'):
    return list(map(lambda word2: (word2, get_cosine_similarity(model, word1, word2)), get_synonyms(word1, lang=lang)))

def get_max_synonym_similarity_between_words(word1, word2, model, lang='spa'):
    synonyms_word1 = get_synonyms(word1, lang = lang)
    synonyms_word2 = get_synonyms(word2, lang = lang)
    synonyms = list(set(synonyms_word1).intersection(synonyms_word2))
    if len(synonyms) !=0:
        max_value = max(synonyms)
    else:
        max_value = 0
    max_value = max_value if max_value else 0
    return max_value

def get_number_of_same_synonyms(word1, word2, lang = 'spa'):
    synonyms_word1 = get_synonyms(word1, lang = lang)
    synonyms_word2 = get_synonyms(word2, lang = lang)
    return len(set(synonyms_word1).intersection(synonyms_word2))

def get_number_of_same_antonyms(word1, word2, lang = 'spa'):
    antonyms_word1 = get_antonyms(word1, lang = lang)
    antonyms_word2 = get_antonyms(word2, lang = lang)
    return len(set(antonyms_word1).intersection(antonyms_word2))

def get_number_synonyms_vs_antonyms(word1, word2, lang = 'spa'):
    antonyms_word1 = get_antonyms(word1, lang = lang)
    synonyms_word1 = get_synonyms(word1, lang = lang)
    antonyms_word2 = get_antonyms(word2, lang = lang)
    synonyms_word2 = get_synonyms(word2, lang = lang)
    return len(set(synonyms_word1).intersection(antonyms_word2)) + len(set(antonyms_word1).intersection(synonyms_word2))

def get_antonyms_with_similarity(word1, model, lang='spa'):
    return list(map(lambda word2: (word2, get_cosine_similarity(model, word1, word2)), get_antonyms(word1, lang=lang)))

def get_max_antonym_similarity_between_words(word1, word2, model, lang='spa'):
    antonyms_word1 = get_antonyms(word1, lang = lang)
    antonyms_word2 = get_antonyms(word2, lang = lang)
    antonyms = list(set(antonyms_word1).intersection(antonyms_word2))
    if len(antonyms) != 0:
        max_value = max(antonyms)
    else:
        max_value = 0
    max_value = max_value if max_value else 0
    return max_value

def get_hypernyms_of_synsets(synsets):
    hypernyms = list(map(lambda x: x.hypernyms(), synsets))
    return list(set(itertools.chain(*hypernyms)))

def get_path_similarity(synset1, synset2):
    similarity = wn.path_similarity(synset1, synset2)
    similarity = similarity if similarity else 0
    return similarity

def get_first_common_hypernym(word1, word2, lang = 'spa'):
    temporal_hypernyms1 = get_hypernyms_of_synsets(wn.synsets(word1, lang=lang))
    temporal_hypernyms2 = get_hypernyms_of_synsets(wn.synsets(word2, lang=lang))
    hypernyms1 = list(set(temporal_hypernyms1))
    hypernyms2 = list(set(temporal_hypernyms2))
    while len(set(hypernyms1).intersection(hypernyms2))==0 and (len(temporal_hypernyms1)!=0 or len(temporal_hypernyms2)!=0):
        hypernyms1 = list(set(hypernyms1 + temporal_hypernyms1))
        hypernyms2 = list(set(hypernyms2 + temporal_hypernyms2))
        temporal_hypernyms1 = get_hypernyms_of_synsets(temporal_hypernyms1)
        temporal_hypernyms2 = get_hypernyms_of_synsets(temporal_hypernyms2)
    if len(set(hypernyms1).intersection(hypernyms2))!=0:
        hypernyms = list(set(hypernyms1).intersection(hypernyms2))
        possible_hypernyms = list(map(lambda x: max(list(map(lambda y: get_path_similarity(x, y), wn.synsets(word1, lang=lang)))) + max(list(map(lambda y: get_path_similarity(x, y), wn.synsets(word2, lang=lang)))),hypernyms))
        max_value = max(possible_hypernyms)
        hypernym = hypernyms[possible_hypernyms.index(max_value)]
        similarity_to_word1 = max(list(map(lambda x: get_path_similarity(hypernyms[possible_hypernyms.index(max_value)],x),wn.synsets(word1, lang=lang))))
        similarity_to_word2 = max(list(map(lambda x: get_path_similarity(hypernyms[possible_hypernyms.index(max_value)],x),wn.synsets(word2, lang=lang))))
    else:
        hypernym = None
        similarity_to_word1 = 0
        similarity_to_word2 = 0
    return hypernym, similarity_to_word1, similarity_to_word2
    
    
def apply_all_metrics_to_words_df(df, model, word2, lang = 'spa'):
    df['similarity'] = df.apply(lambda x: get_cosine_similarity(model, x['words'], word2), axis=1)
    df['number_same_synonyms'] = df.apply(lambda x: get_number_of_same_synonyms(x['words'], word2, lang= lang), axis=1)
    df['number_same_antonyms'] = df.apply(lambda x: get_number_of_same_antonyms(x['words'], word2, lang= lang), axis=1)
    df['synonyms_vs_antonyms'] = df.apply(lambda x: get_number_synonyms_vs_antonyms(x['words'], word2, lang= lang), axis=1)
    df['max_similarity_synonym'] = df.apply(lambda x: get_max_synonym_similarity_between_words(x['words'], word2, model, lang=lang), axis=1)
    df['max_similarity_antonym'] = df.apply(lambda x: get_max_antonym_similarity_between_words(x['words'], word2, model, lang=lang), axis=1)
    df['distance_common_hypernym'] =  df.apply(lambda x: get_first_common_hypernym(x['words'], word2, lang= lang)[1], axis=1)
    df['distance_common_hypernym_from_word'] =  df.apply(lambda x: get_first_common_hypernym(x['words'], word2, lang= lang)[2], axis=1)
    return df

In [3]:
#Grado similaridad
#Numero de sinonimos en común
#Número de antónimos en común
#Relación con los sinonimos
#Relación con los antónimos
#Familia semántica (Hiperonimos)
#Número de palabras entre la pista y la evaluada(s)

In [124]:
data = pd.DataFrame(words, columns = ["words"])

In [129]:
sample_data = data.sample(25)

In [130]:
sample_data

,words
211,látigo
55,cabo
192,hierba
241,médico
243,mesa
301,pila
188,gusano
159,fiesta
132,dinosaurio
160,figura


In [136]:
apply_all_metrics_to_words_df(sample_data, model, 'dinero')

,words,similarity,number_same_synonyms,number_same_antonyms,synonyms_vs_antonyms,max_similarity_synonym,max_similarity_antonym,distance_common_hypernym,distance_common_hypernym_from_word
211,látigo,0.228567,0,0,0,0,0,0.111111,0.166667
55,cabo,0.188124,0,0,0,0,0,0.200000,0.166667
192,hierba,0.132030,0,0,0,0,0,0.142857,0.250000
241,médico,0.152476,0,0,0,0,0,0.125000,0.200000
243,mesa,0.127269,0,0,0,0,0,0.142857,0.166667
301,pila,0.225776,0,0,0,0,0,0.250000,0.200000
188,gusano,0.181176,0,0,0,0,0,0.111111,0.166667
159,fiesta,0.159714,0,0,0,0,0,0.250000,0.200000
132,dinosaurio,0.134965,0,0,0,0,0,0.076923,0.166667
160,figura,0.031461,0,0,0,0,0,0.250000,0.200000


In [74]:
fasttext.util.download_model('es', if_exists='ignore')  # English

'cc.es.300.bin'

In [100]:
%%capture
FAST_TEXT_MODEL = "cc.es.300.bin" # Model name in fasttext


ft = fasttext.load_model(FAST_TEXT_MODEL)

In [5]:
ft.get_nearest_neighbors('life')

[(0.6984261274337769, 'life-'),
 (0.6904193162918091, 'lives'),
 (0.6745699644088745, 'life.But'),
 (0.6679530143737793, 'life.It'),
 (0.6632518768310547, 'life.The'),
 (0.6621710062026978, 'life.In'),
 (0.6610385775566101, 'life.Now'),
 (0.652584969997406, 'life.So'),
 (0.6445083022117615, 'life.That'),
 (0.643845796585083, 'life--and')]

In [101]:
model = load_facebook_model(FAST_TEXT_MODEL)
#model.wv.most_similar(positive=['red', 'green'])

In [53]:
model.wv.similarity('casa', 'hogar')

0.61479205

In [67]:
a = list(map(lambda x: x.lemma_names('spa'), wn.synsets('casa', lang='spa')))
set(itertools.chain(*a))

['hogar',
 'domiciliación',
 'morada',
 'piso',
 'país',
 'residencia',
 'empresa',
 'alojamiento',
 'casa',
 'trincheras',
 'firma',
 'familia',
 'domicilio',
 'pensión',
 'vivienda',
 'habitáculo']

In [19]:
get_antonyms('bien', lang='spa')

['mal', 'maldad']

In [27]:
get_number_synonyms_vs_antonyms('bien', 'mal', lang='spa')

3

In [38]:
first = wn.synset('apple.n.01')
second = wn.synset('banana.n.01')

0.07142857142857142

In [41]:
fist = first.hypernyms()

In [55]:
fist = list(map(lambda x: x.hypernyms(),list(set(itertools(*fist)))))

TypeError: 'module' object is not callable

In [86]:
get_first_common_hypernym('manzana', 'bien', lang ='spa')

(Synset('physical_entity.n.01'), 0.14285714285714285, 0.2)

In [60]:
get_hypernyms_of_synsets(wn.synsets('manzana', lang='spa'))

[Synset('edible_fruit.n.01'), Synset('pome.n.01'), Synset('apple.n.01')]

In [81]:
wn.synsets('bien', lang='spa')

[Synset('well.r.13'),
 Synset('well.r.06'),
 Synset('well.r.12'),
 Synset('good.s.17'),
 Synset('all_right.s.01'),
 Synset('commodity.n.01'),
 Synset('good.n.03'),
 Synset('sake.n.01'),
 Synset('good.n.01'),
 Synset('personal_property.n.01')]

In [27]:
wn.synsets('manzana', lang='spa')

[Synset('apple.n.01'), Synset('eating_apple.n.01')]

In [28]:
wn.synsets('pera', lang ='spa')

[Synset('goatee.n.01'), Synset('pear.n.01'), Synset('pear.n.02')]

In [82]:
wn.path_similarity(wn.synset('physical_entity.n.01'), wn.synset('well.r.13'))